## Basic Python - Project <a id='intro'></a>

## Introduction <a id='intro'></a>
In this project, you will work with data from the entertainment industry. You will study a dataset with records on movies and shows. The research will focus on the "Golden Age" of television, which began in 1999 with the release of *The Sopranos* and is still ongoing.

The aim of this project is to investigate how the number of votes a title receives impacts its ratings. The assumption is that highly-rated shows (we will focus on TV shows, ignoring movies) released during the "Golden Age" of television also have the most votes.

### Stages 
Data on movies and shows is stored in the `/datasets/movies_and_shows.csv` file. There is no information about the quality of the data, so you will need to explore it before doing the analysis.

First, you'll evaluate the quality of the data and see whether its issues are significant. Then, during data preprocessing, you will try to account for the most critical problems.
 
Your project will consist of three stages:
 1. Data overview
 2. Data preprocessing
 3. Data analysis

## Stage 1. Data overview <a id='data_review'></a>

Open and explore the data.

You'll need `pandas`, so import it.

In [2]:
# importing pandas
import pandas as pd

Read the `movies_and_shows.csv` file from the `datasets` folder and save it in the `df` variable:

In [3]:
# reading the files and storing them to df
df = pd.read_csv('/datasets/movies_and_shows.csv')

Print the first 10 table rows:

In [4]:
# obtaining the first 10 rows from the df table
# hint: you can use head() and tail() in Jupyter Notebook without wrapping them into print()
print(df.head(10))

              name                      Character   r0le        TITLE   Type  \
0   Robert De Niro                  Travis Bickle  ACTOR  Taxi Driver  MOVIE   
1     Jodie Foster                  Iris Steensma  ACTOR  Taxi Driver  MOVIE   
2    Albert Brooks                            Tom  ACTOR  Taxi Driver  MOVIE   
3    Harvey Keitel        Matthew 'Sport' Higgins  ACTOR  Taxi Driver  MOVIE   
4  Cybill Shepherd                          Betsy  ACTOR  Taxi Driver  MOVIE   
5      Peter Boyle                         Wizard  ACTOR  Taxi Driver  MOVIE   
6   Leonard Harris      Senator Charles Palantine  ACTOR  Taxi Driver  MOVIE   
7   Diahnne Abbott                Concession Girl  ACTOR  Taxi Driver  MOVIE   
8      Gino Ardito             Policeman at Rally  ACTOR  Taxi Driver  MOVIE   
9  Martin Scorsese  Passenger Watching Silhouette  ACTOR  Taxi Driver  MOVIE   

   release Year              genres  imdb sc0re  imdb v0tes  
0          1976  ['drama', 'crime']         8.2    808582

Obtain the general information about the table with one command:

In [5]:
# obtaining general information about the data in df
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85579 entries, 0 to 85578
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0      name       85579 non-null  object 
 1   Character     85579 non-null  object 
 2   r0le          85579 non-null  object 
 3   TITLE         85578 non-null  object 
 4     Type        85579 non-null  object 
 5   release Year  85579 non-null  int64  
 6   genres        85579 non-null  object 
 7   imdb sc0re    80970 non-null  float64
 8   imdb v0tes    80853 non-null  float64
dtypes: float64(2), int64(1), object(6)
memory usage: 5.9+ MB
None


The table contains nine columns. The majority store the same data type: object. The only exceptions are `'release Year'` (int64 type), `'imdb sc0re'` (float64 type) and `'imdb v0tes'` (float64 type). Scores and votes will be used in our analysis, so it's important to verify that they are present in the dataframe in the appropriate numeric format. Three columns (`'TITLE'`, `'imdb sc0re'` and `'imdb v0tes'`) have missing values.

According to the documentation:
- `'name'` — actor/director's name and last name
- `'Character'` — character played (for actors)
- `'r0le '` — the person's contribution to the title (it can be in the capacity of either actor or director)
- `'TITLE '` — title of the movie (show)
- `'  Type'` — show or movie
- `'release Year'` — year when movie (show) was released
- `'genres'` — list of genres under which the movie (show) falls
- `'imdb sc0re'` — score on IMDb
- `'imdb v0tes'` — votes on IMDb

We can see three issues with the column names:
1. Some names are uppercase, while others are lowercase.
2. There are names containing whitespace.
3. A few column names have digit '0' instead of letter 'o'. 


### Conclusions <a id='data_review_conclusions'></a> 

Each row in the table stores data about a movie or show. The columns can be divided into two categories: the first is about the roles held by different people who worked on the movie or show (role, name of the actor or director, and character if the row is about an actor); the second category is information about the movie or show itself (title, release year, genre, imdb figures).

It's clear that there is sufficient data to do the analysis and evaluate our assumption. However, to move forward, we need to preprocess the data.

## Stage 2. Data preprocessing <a id='data_preprocessing'></a>
Correct the formatting in the column headers and deal with the missing values. Then, check whether there are duplicates in the data.

In [6]:
# the list of column names in the df table
column_names = df.columns.tolist()
print(column_names)

['   name', 'Character', 'r0le', 'TITLE', '  Type', 'release Year', 'genres', 'imdb sc0re', 'imdb v0tes']


Change the column names according to the rules of good style:
* If the name has several words, use snake_case
* All characters must be lowercase
* Remove whitespace
* Replace zero with letter 'o'

In [7]:
# renaming columns
column_names = [column.strip().lower().replace(' ', '_').replace('0', 'o') for column in column_names]
df.columns = column_names

Check the result. Print the names of the columns once more:

In [8]:
# checking result: the list of column names
print(column_names)

['name', 'character', 'role', 'title', 'type', 'release_year', 'genres', 'imdb_score', 'imdb_votes']


### Missing values <a id='missing_values'></a>
First, find the number of missing values in the table. To do so, combine two `pandas` methods:

In [9]:
# calculating missing values
print(df.isna().sum())

name               0
character          0
role               0
title              1
type               0
release_year       0
genres             0
imdb_score      4609
imdb_votes      4726
dtype: int64


Not all missing values affect the research: the single missing value in `'title'` is not critical. The missing values in columns `'imdb_score'` and `'imdb_votes'` represent around 6% of all records (4,609 and 4,726, respectively, of the total 85,579). This could potentially affect our research. To avoid this issue, we will drop rows with missing values in the `'imdb_score'` and `'imdb_votes'` columns.

In [10]:
# dropping rows where columns with title, scores and votes have missing values
df = df.dropna(axis='rows')

Make sure the table doesn't contain any more missing values. Count the missing values again.

In [11]:
# counting missing values
print(df.isna().sum())

name            0
character       0
role            0
title           0
type            0
release_year    0
genres          0
imdb_score      0
imdb_votes      0
dtype: int64


### Duplicates <a id='duplicates'></a>
Find the number of duplicate rows in the table using one command:

In [12]:
# counting duplicate rows
print(df.duplicated().sum())

6994


Review the duplicate rows to determine if removing them would distort our dataset.

In [13]:
# Produce table with duplicates (with original rows included) and review last 5 rows
df = df.sort_values(by='name')
duplicates_with_originals = df[df.duplicated(keep=False)]
print(duplicates_with_originals.tail(5))

                    name  character   role                 title   type  \
77975  Žø?tar G??kseven  Ali Seker  ACTOR              The Club   SHOW   
76500          ‚§‘ý?‘ñ    unknown  ACTOR          Zero to Hero  MOVIE   
84278          ‚§‘ý?‘ñ    unknown  ACTOR          Zero to Hero  MOVIE   
76630       ‡??†ññ‡®?‘ª­    unknown  ACTOR  Creator's File: GOLD   SHOW   
84408       ‡??†ññ‡®?‘ª­    unknown  ACTOR  Creator's File: GOLD   SHOW   

       release_year               genres  imdb_score  imdb_votes  
77975          2021  ['drama', 'comedy']         7.9      7810.0  
76500          2021   ['drama', 'sport']         6.7       842.0  
84278          2021   ['drama', 'sport']         6.7       842.0  
76630          2021  ['drama', 'comedy']         6.6        22.0  
84408          2021  ['drama', 'comedy']         6.6        22.0  


There are two clear duplicates in the printed rows. We can safely remove them.
Call the `pandas` method for getting rid of duplicate rows:

In [20]:
# removing duplicate rows
df = df.drop_duplicates()
print(df.tail(5))

                      name  character      role                  title   type  \
76500            ‚§‘ý?‘ñ    unknown     ACTOR           Zero to Hero  MOVIE   
37611            †??‘æú‘?›    unknown  DIRECTOR  The Rise of Phoenixes   SHOW   
11835            †?‘?­Š?¯  ‘§®‚??Š?     ACTOR        The Fierce Wife   SHOW   
76630         ‡??†ññ‡®?‘ª­    unknown     ACTOR   Creator's File: GOLD   SHOW   
7297   ‰??‰ÿ??? ??‰¦ª??    unknown  DIRECTOR              Octonauts   SHOW   

       release_year                                       genres  imdb_score  \
76500          2021                           ['drama', 'sport']         6.7   
37611          2018              ['drama', 'romance', 'history']         8.4   
11835          2010                                    ['drama']         6.8   
76630          2021                          ['drama', 'comedy']         6.6   
7297           2010  ['action', 'animation', 'comedy', 'family']         7.6   

       imdb_votes  
76500       

Check for duplicate rows once more to make sure you have removed all of them:

In [21]:
# checking for duplicates
print(df.duplicated().sum())

0


Now get rid of implicit duplicates in the `'type'` column. For example, the string `'SHOW'` can be written in different ways. These kinds of errors will also affect the result.

Print a list of unique `'type'` names, sorted in alphabetical order. To do so:
* Retrieve the intended dataframe column 
* Apply a sorting method to it
* For the sorted column, call the method that will return all unique column values

In [22]:
# viewing unique type names
display(sorted(df['type'].unique()))

['MOVIE',
 'SHOW',
 'movies',
 'shows',
 'the movie',
 'tv',
 'tv series',
 'tv show',
 'tv shows']

Look through the list to find implicit duplicates of `'show'` (`'movie'` duplicates will be ignored since the assumption is about shows). These could be names written incorrectly or alternative names of the same genre.

You will see the following implicit duplicates:
* `'shows'`
* `'SHOW'`
* `'tv show'`
* `'tv shows'`
* `'tv series'`
* `'tv'`

To get rid of them, declare the function `replace_wrong_show()` with two parameters: 
* `wrong_shows_list=` — the list of duplicates
* `correct_show=` — the string with the correct value

The function should correct the names in the `'type'` column from the `df` table (i.e., replace each value from the `wrong_shows_list` list with the value in `correct_show`).

In [26]:
# function for replacing implicit duplicates
def replace_wrong_show(wrong_shows_list, correct_show):
    df['type'].replace(wrong_shows_list, correct_show)

Call `replace_wrong_show()` and pass it arguments so that it clears implicit duplicates and replaces them with `SHOW`:

In [32]:
# removing implicit duplicates
def replace_wrong_show(wrong_shows_list, correct_show):
    for show in wrong_shows_list:
        df['type'] = df['type'].replace(show, correct_show)
    return df

replace_wrong_show(['shows', 'tv show', 'tv shows', 'tv series', 'tv'], 'SHOW')

,name,character,role,title,type,release_year,genres,imdb_score,imdb_votes
18281,Michael Hayden,Self - Former NSA and CIA Director,ACTOR,We Steal Secrets: The Story of WikiLeaks,MOVIE,2013,['documentation'],6.9,7934.0
62571,'Jeeva' Ravi,Dr. Vasanth,ACTOR,Andhaghaaram,MOVIE,2020,"['thriller', 'horror']",7.6,3746.0
34335,'Weird Al' Yankovic,Himself,ACTOR,Sandy Wexler,MOVIE,2017,['comedy'],5.2,17039.0
66906,'Weird Al' Yankovic,Self,ACTOR,The Sparks Brothers,MOVIE,2021,"['documentation', 'music', 'comedy', 'history']",7.8,4663.0
59804,21 Savage,21 Savage,ACTOR,The Trap,MOVIE,2019,['comedy'],4.3,1625.0
...,...,...,...,...,...,...,...,...,...
76500,‚§‘ý?‘ñ,unknown,ACTOR,Zero to Hero,MOVIE,2021,"['drama', 'sport']",6.7,842.0
37611,†??‘æú‘?›,unknown,DIRECTOR,The Rise of Phoenixes,SHOW,2018,"['drama', 'romance', 'history']",8.4,1492.0
11835,†?‘?­Š?¯,‘§®‚??Š?,ACTOR,The Fierce Wife,SHOW,2010,['drama'],6.8,95.0
76630,‡??†ññ‡®?‘ª­,unknown,ACTOR,Creator's File: GOLD,SHOW,2021,"['drama', 'comedy']",6.6,22.0


Make sure the duplicate names are removed. Print the list of unique values from the `'type'` column:

In [34]:
# viewing unique genre names
display(sorted(df['type'].unique()))

['MOVIE', 'SHOW', 'movies', 'the movie']

### Conclusions <a id='data_preprocessing_conclusions'></a>
We detected three issues with the data:

- Incorrect header styles
- Missing values
- Duplicate rows and implicit duplicates

The headers have been cleaned up to make processing the table simpler.

All rows with missing values have been removed. 

The absence of duplicates will make the results more precise and easier to understand.

Now we can move on to our analysis of the prepared data.

## Stage 3. Data analysis <a id='hypotheses'></a>

Based on the previous project stages, you can now define how the assumption will be checked. Calculate the average amount of votes for each score (this data is available in the `imdb_score` and `imdb_votes` columns), and then check how these averages relate to each other. If the averages for shows with the highest scores are bigger than those for shows with lower scores, the assumption appears to be true.

Based on this, complete the following steps:

- Filter the dataframe to only include shows released in 1999 or later.
- Group scores into buckets by rounding the values of the appropriate column (a set of 1-10 integers will help us make the outcome of our calculations more evident without damaging the quality of our research).
- Identify outliers among scores based on their number of votes, and exclude scores with few votes.
- Calculate the average votes for each score and check whether the assumption matches the results.

To filter the dataframe and only include shows released in 1999 or later, you will take two steps. First, keep only titles published in 1999 or later in our dataframe. Then, filter the table to only contain shows (movies will be removed).

In [35]:
# using conditional indexing modify df so it has only titles released after 1999 (with 1999 included)
# give the slice of dataframe new name
shows_1999_or_later = df[df['release_year'] >= 1999]

In [36]:
# repeat conditional indexing so df has only shows (movies are removed as result)
only_shows = shows_1999_or_later[shows_1999_or_later['type'] == 'SHOW']

The scores that are to be grouped should be rounded. For instance, titles with scores like 7.8, 8.1, and 8.3 will all be placed in the same bucket with a score of 8.

In [37]:
# rounding column with scores
only_shows['imdb_score'] = only_shows['imdb_score'].round().astype(int)

/tmp/ipykernel_27/1018010210.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  only_shows['imdb_score'] = only_shows['imdb_score'].round().astype(int)


It is now time to identify outliers based on the number of votes.

In [40]:
# Use groupby() for scores and count all unique values in each group, print the result
only_shows.groupby('imdb_score').agg({'name': 'nunique'})

,name
imdb_score,
2,24
3,27
4,174
5,581
6,2365
7,4342
8,4194
9,542
10,8


Based on the aggregation performed, it is evident that scores 2 (24 voted shows), 3 (27 voted shows), and 10 (only 8 voted shows) are outliers. There isn't enough data for these scores for the average number of votes to be meaningful.

To obtain the mean numbers of votes for the selected scores (we identified a range of 4-9 as acceptable), use conditional filtering and grouping.

In [41]:
# filter dataframe using two conditions (scores to be in the range 4-9)
filtered_scores = only_shows[(only_shows['imdb_score'] >= 4) & (only_shows['imdb_score'] <= 9)]

# group scores and corresponding average number of votes, reset index and print the result
avg_votes_per_score = filtered_scores.groupby('imdb_score')['imdb_votes'].mean().reset_index()
avg_votes_per_score

,imdb_score,imdb_votes
0,4,5277.583333
1,5,3143.942568
2,6,3481.717322
3,7,8727.068211
4,8,30299.460967
5,9,126904.109515


Now for the final step! Round the column with the averages, rename both columns, and print the dataframe in descending order.

In [42]:
# round column with averages
avg_votes_per_score['imdb_votes'] = avg_votes_per_score['imdb_votes'].round()
# rename columns
avg_votes = avg_votes_per_score.rename(columns= {'imdb_score': 'Score', 'imdb_votes': 'Avg Votes'})
# print dataframe in descending order
avg_votes = avg_votes[['Score', 'Avg Votes']].sort_values(by='Score', ascending=False)
avg_votes

,Score,Avg Votes
5,9,126904.0
4,8,30299.0
3,7,8727.0
2,6,3482.0
1,5,3144.0
0,4,5278.0


The assumption macthes the analysis: the shows with the top 3 scores have the most amounts of votes.

## Conclusion <a id='hypotheses'></a>

The research done confirms that highly-rated shows released during the "Golden Age" of television also have the most votes. While shows with score 4 have more votes than ones with scores 5 and 6, the top three (scores 7-9) have the largest number. The data studied represents around 94% of the original set, so we can be confident in our findings.